In [30]:
from tqdm.notebook import tqdm
from PIL import Image
import os
import numpy as np
import torch
import pandas as pd
import matplotlib.pyplot as plt
from skimage.io import imread

# Instructions

1. Change directories
2. Run notebook

The notebook will resume if it is partially done.

In [53]:
%%time

data_dir = "./data/train2017"
out_dir = "./data/train2017_processed/originals"
out_dir_masked = "./data/train2017_processed/masked"
image_dim = 224

try:
    os.mkdir(out_dir)
    os.mkdir(out_dir_masked)
except FileExistsError:
    pass

annos = pd.read_pickle("./annotations_train2017.pickle")

# For resuming partially done
done = os.listdir(out_dir)
done_masked = os.listdir(out_dir_masked)

for image_filename in tqdm(os.listdir(data_dir), "Processing Images"):
    
    image_id = int(image_filename.split(".")[0])
    
    masked_out_paths = [str(image_id) + f"_masked_{idx}.npy" for idx in range(0,5)]
    full_out_path = str(image_id) + ".npy"
    exists = True
    if full_out_path not in done:
        exists = False
    
    for out_path in masked_out_paths:
        if out_path not in done_masked:
            exists = False
            break
    if exists:
        continue
    
    image_path = os.path.join(data_dir, image_filename)
    im = Image.open(image_path)
    im = im.resize((image_dim, image_dim))
    
    full_image = np.array(im) / 255
    


    with open(os.path.join(out_dir, full_out_path), 'wb') as f:
        np.save(f, full_image)

    for idx in range(5):

        anno = annos[image_id][idx]

        temp = full_image.copy()
        temp[anno['coord_start'][0]:anno['coord_end'][0],anno['coord_start'][1]:anno['coord_end'][1]] = 0

        with open(os.path.join(out_dir_masked, masked_out_paths[idx]), 'wb') as f:
            np.save(f, temp)

KeyboardInterrupt: 